***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-77ea  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [9]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  4 15:22 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [11]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [12]:
spark

In [13]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ofek_alon_project'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.startswith('multi'):
        paths.append(full_path+b.name)

In [14]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sc.addFile("/home/dataproc/pickle_handler.py")
sc.addFile("/home/dataproc/preprocess_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

23/01/04 16:46:00 WARN org.apache.spark.SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.
23/01/04 16:46:00 WARN org.apache.spark.SparkContext: The path /home/dataproc/tokenizer.py has been added already. Overwriting of added paths is not supported in the current version.
23/01/04 16:46:00 WARN org.apache.spark.SparkContext: The path /home/dataproc/pickle_handler.py has been added already. Overwriting of added paths is not supported in the current version.


In [15]:
from preprocessor_gcp import PreProcessor
from pickle_handler import PickleHandler
from inverted_index_gcp import *

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.3
      /_/

Using Python version 3.8.15 (default, Nov 22 2022 08:46:39)
Spark context Web UI available at http://cluster-77ea-m.us-central1-a.c.assignment3-370316.internal:46539
Spark context available as 'sc' (master = yarn, app id = application_1672845681143_0008).
SparkSession available as 'spark'.


In [16]:
pickle_handler = PickleHandler()
preprocessor = PreProcessor(spark)

In [17]:
# indices = [('text', False), ('text', True), ('title', False), ('anchor_text', False)]
indices = [('title', False)]

In [18]:
def reduce_word_counts(word_counts):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
      unsorted_pl: list of tuples
        A list of (wiki_id, tf) tuples
    Returns:
    --------
      list of tuples
        A sorted posting list.
    '''
    return sorted(word_counts)

In [19]:
NUM_BUCKETS = 124


def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()


def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

In [20]:
def partition_postings_and_write(postings, bucket_name: str, index_name: str):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
      postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and
        offsets its posting list was written to. See `write_a_posting_list` for
        more details.
    '''
    rdd = postings.map(lambda x: (token2bucket_id(x[0]), [x])).reduceByKey(lambda x, y: x + y)
    posting_locs_rdd = rdd.map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name, index_name))
    return posting_locs_rdd

In [21]:
for index in indices:
    index_type, stemming = index
    index_name = index_type + "_with_stemming" if stemming else index_type
    anchor = index_type == 'anchor_text'
    doc_text_pairs, num_of_docs = preprocessor._read_gcp_files(paths, index_type)
    if index_type == 'title':
        title_id_dict = doc_text_pairs.map(lambda x: {x['id']: x[index_type]}).collect()
        pickle_handler.write_pickle_file('title.pkl', title_id_dict)
    word_counts = doc_text_pairs.flatMap(lambda x: preprocessor._word_count(text=x[index_type], doc_id=x['id'], stemming=stemming))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    posting_locs_list = partition_postings_and_write(postings, bucket_name, index_name).collect()
    super_posting_locs = preprocessor.get_super_posting_locs(bucket_name)
    inverted_index = InvertedIndex()
    inverted_index.posting_locs = super_posting_locs
    inverted_index.df = preprocessor.calculate_df(postings)
    if index_type == 'text':
        inverted_index.idf = preprocessor.calculate_idf(postings, num_of_docs) # to be saved seperately
        inverted_index.num_of_docs = num_of_docs
        tf_idf_dict = preprocessor.calculate_tf_idf(postings, inverted_index.idf)
        pickle_handler.write_pickle_file(f'tf_idf_{index_name}.pkl', tf_idf_dict)
    inverted_index.write_index('.', 'index')
    index_src = "index.pkl"
    index_dst = f'gs://{bucket_name}/postings_gcp_{index_name}/{index_src}'
    !gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 66.7 MiB/ 66.7 MiB]                                                
Operation completed over 1 objects/66.7 MiB.                                     


In [24]:
inverted_index.read_index(f'gs://{bucket_name}/postings_gcp_{index_name}', 'index')

FileNotFoundError: [Errno 2] No such file or directory: 'gs:/ofek_alon_project/postings_gcp_title/index.pkl'

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [19]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  edges = pages.flatMap(lambda x: [(x['id'], row['id']) for row in x['anchor_text']]).distinct()
  vertices = pages.flatMap(lambda x: [x['id']] + [row['id'] for row in x['anchor_text']]).distinct().map(lambda x: Row(id=x))
  return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9917.889528786583|
|  10568| 5387.609470096125|
|  32927| 5284.298441243094|
|  30680|5130.3860058361915|
|5843419| 4959.648354746545|
|  68253| 4771.279909629093|
|  31717| 4488.233081199123|
|  11867| 4148.154882176616|
|  14533|3998.1437395714433|
| 645042|3533.1092975401457|
|  17867|3247.4607632514612|
|5042916|2993.2014451081495|
|4689264|2983.5765511544087|
|  14532| 2935.978529024251|
|  25391|2904.7648285829773|
|   5405|2892.6298433630327|
|4764461|2835.5565581932296|
|  15573| 2785.033493615905|
|   9316|2783.2072557286942|
|8569916| 2776.450966760946|
+-------+------------------+
only showing top 20 rows



In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [22]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [23]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://alonofir/postings_gcp


In [1]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 16
print(f'I used {cost} USD credit during the course of this assignment')

I used 16 USD credit during the course of this assignment


**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [20]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [21]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag